In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rc('pdf', fonttype=42) # Avoid type 3 fonts

sns.set_theme(style="white")
sns.set_palette(sns.color_palette("tab10"))

In [ ]:
BASE_DIR = '../ios'
DATA_DIR = os.path.join(BASE_DIR, 'server/data')

FILES = [
    'CHANGE-ME.json',
    'CHANGE-ME.json',
]


def get_device_info(device_name):
    with open(os.path.join(BASE_DIR, 'server/models.json')) as f:
        return json.load(f)[device_name]


def read_data(path):
    with open(path) as f:
        d = json.load(f)
        
    info = get_device_info(d['device'])
    
    return pd.DataFrame(data={
        'Device': info['name'],
        'OS Version': d['version'],
        'Chip': info['chip'],
        'Configuration': f"{info['name']} ({d['version']})",
        'Measurement': [float(x) * 1000 for x in d['executionTimeSeconds']],
    })


def read_all_data():
    for f in FILES:
        path = os.path.join(DATA_DIR, f)
        x = read_data(path)
        yield x
        
df = pd.concat(read_all_data())
df.sample(10)

In [ ]:
#
# Box plot with the iOS ECDH execution times
#

fig, ax = plt.subplots()
fig.set_size_inches((5.5, 2.5))

sns.boxplot(
    data=df,
    y='Configuration',
    x='Measurement',
    hue='Chip',
    ax=ax,
    showfliers = False,
    dodge = False,
    
)

ax.set_xlabel("Operation duration [ms]")
ax.set_xlim((5.5, 18.5))

ax.set_ylabel("Configuration")

ax.grid(ls='--')

ax.legend(loc='lower right',ncol=2)

fig.tight_layout()
fig.savefig("output/ios-operation-box-plot.pdf", dpi=300, bbox_inches='tight')

In [ ]:
#
# Intermediate dataset with the p10 execution times
#

measured = df.groupby('Device').quantile(0.1)
measured.to_csv('output/ios-p10-times.csv')
measured